In [1]:
import importlib
import pandas as pd
from pprint import pprint
from core.clean_data import DataCleaner
from core.join_data import JoinPrepper
from core.utils import full_file_path
from core.get_data import get_solicitacoes, get_usuarios

In [2]:
pasta_dados = 'original_data'

In [3]:
df_users = get_usuarios()
df_solicit = get_solicitacoes()

In [4]:
clean_data = DataCleaner()

In [5]:
df_solicit = clean_data(df_solicit)

Droped 1 all empty rows
Null report:
{'A Solicitação foi': {'prop_nulos': 0.014744682525112119, 'qtd_nulos': 2811},
 'Anexo Resposta': {'prop_nulos': 0.8268352172886779, 'qtd_nulos': 157632},
 'Anexo Solicitação': {'prop_nulos': 0.9290566235673634, 'qtd_nulos': 177120},
 'Anexo do Recurso': {'prop_nulos': 0.9776338220252302, 'qtd_nulos': 186381},
 'Anexo do Recurso.1': {'prop_nulos': 0.9855438117967951, 'qtd_nulos': 187889},
 'Anexo do Recurso.2': {'prop_nulos': 0.9932387421647565, 'qtd_nulos': 189356},
 'Classificação da Resposta': {'prop_nulos': 0.9468803273099216,
                               'qtd_nulos': 180518},
 'Classificação da Resposta.1': {'prop_nulos': 0.9829106454404783,
                                 'qtd_nulos': 187387},
 'Classificação da Resposta.2': {'prop_nulos': 0.9924781662251829,
                                 'qtd_nulos': 189211},
 'Classificação do Motivo de Recurso': {'prop_nulos': 0.9472002937396732,
                                        'qtd_nulos': 18

In [6]:
df_users = clean_data(df_users)

Droped 2 all empty rows
Null report:
{'Cargo responsável': {'prop_nulos': 0.9528534938975459, 'qtd_nulos': 181672},
 'Data Nascimento': {'prop_nulos': 0.3486030179218613, 'qtd_nulos': 66465},
 'Escolaridade': {'prop_nulos': 0.33181405741079717, 'qtd_nulos': 63264},
 'Estado': {'prop_nulos': 0.13377145824264008, 'qtd_nulos': 25505},
 'Município': {'prop_nulos': 0.18060326967759532, 'qtd_nulos': 34434},
 'Número de Controle': {'prop_nulos': 0.0, 'qtd_nulos': 0},
 'País': {'prop_nulos': 0.004332296589234295, 'qtd_nulos': 826},
 'Profissão': {'prop_nulos': 0.3723572204069002, 'qtd_nulos': 70994},
 'Sexo': {'prop_nulos': 0.2793754359832373, 'qtd_nulos': 53266},
 'Tipo Pessoa': {'prop_nulos': 3.671437787486691e-05, 'qtd_nulos': 7},
 'Tipo de Instituição': {'prop_nulos': 0.9410262193107137, 'qtd_nulos': 179417},
 'Área de Atuação': {'prop_nulos': 0.9439056755183284, 'qtd_nulos': 179966}}
Droped following columns with > 0.9 nuls:
['Área de Atuação', 'Tipo de Instituição', 'Cargo responsável']


In [7]:
prep_join = JoinPrepper()

In [8]:
df_users = prep_join(df_users)
df_solicit= prep_join(df_solicit)

In [9]:
df_users['Número de Controle'] = df_users['Número de Controle'].apply(lambda x: x + '0')

In [10]:
merged = pd.merge(df_solicit, df_users, how='left', on='Número de Controle', suffixes=('_solicitacao', '_usuario'), indicator=True)

In [11]:
merged['_merge'].value_counts()

both          190643
left_only          1
right_only         0
Name: _merge, dtype: int64

In [14]:
sem_join = merged['_merge']=='left_only'

In [15]:
merged[sem_join]

,Data do Pedido,Número de Controle,Forma do Pedido,Orgão/Entidade,SIC,Pedido,Síntese,Forma de recebimento da resposta,Situação do pedido,Natureza da Solicitação,...,Anexo Resposta,Tipo Pessoa,Sexo,Data Nascimento,Município,Estado,País,Escolaridade,Profissão,_merge
45001,2015-07-30 14:19:59,680,NaN,Secretaria de Desenvolvimento Econômico,Junta Comercial do Estado de São Paulo - JUCESP,Como adquirir o PPP de uma empresa que foi ext...,Como adquirir o PPP de uma empresa que foi ext...,Correspondência eletrônica (e-mail),Encerrada,Pública,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [18]:
merged = merged[~sem_join].copy().reset_index(drop=True)

In [20]:
path_salvar = full_file_path('dados_merged.csv', 'generated_data')
merged.to_csv(path_salvar, encoding='utf-8')